In [1]:
from __future__ import annotations

import gzip
import pathlib
import pickle
import random
from pathlib import Path

import awkward as ak
import correctionlib
import numpy as np
import uproot
from coffea.analysis_tools import Weights
from coffea.lookup_tools.dense_lookup import dense_lookup
from coffea.nanoevents.methods import vector
from coffea.nanoevents.methods.base import NanoEventsArray
from coffea.nanoevents.methods.nanoaod import FatJetArray, JetArray, MuonArray

# Important Run3 start of Run
FirstRun_2022C = 355794
FirstRun_2022D = 357487
LastRun_2022D = 359021
FirstRun_2022E = 359022
LastRun_2022F = 362180

In [2]:
pog_correction_path = "/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/"
pog_jsons = {
    "muon": ["MUO", "muon_Z.json.gz"],
    "electron": ["EGM", "electron.json.gz"],
    "pileup": ["LUM", "puWeights.json.gz"],
    "fatjet_jec": ["JME", "fatJet_jerc.json.gz"],
    "jet_jec": ["JME", "jet_jerc.json.gz"],
    "jmar": ["JME", "jmar.json.gz"],
    "jetveto": ["JME", "jetvetomaps.json.gz"],
    "btagging": ["BTV", "btagging.json.gz"],
}

def get_pog_json(obj: str, year: str) -> str:
    try:
        pog_json = pog_jsons[obj]
    except:
        print(f"No json for {obj}")

    year = {
        "2016": "2016postVFP_UL",
        "2016APV": "2016preVFP_UL",  
        "2017": "2017_UL",  
        "2018": "2018_UL",  
        "2022": "2022_Summer22",
        "2022EE": "2022_Summer22EE",
        "2023": "2023_Summer23",
        "2023BPix": "2023_Summer23BPix",
    }[year]

    return f"{pog_correction_path}/POG/{pog_json[0]}/{year}/{pog_json[1]}"

In [3]:
year="2016APV"
cset = correctionlib.CorrectionSet.from_file(get_pog_json("jetveto", year))
list(cset.keys())

['Summer19UL16_V1']

In [4]:
year="2016"
cset = correctionlib.CorrectionSet.from_file(get_pog_json("jetveto", year))
list(cset.keys())

['Summer19UL16_V1']

In [5]:
year="2017"
cset = correctionlib.CorrectionSet.from_file(get_pog_json("jetveto", year))
list(cset.keys())

['Summer19UL17_V1']

In [6]:
year="2018"
cset = correctionlib.CorrectionSet.from_file(get_pog_json("jetveto", year))
list(cset.keys())

['Summer19UL18_V1']

In [7]:
year="2016APV"
cset = correctionlib.CorrectionSet.from_file(get_pog_json("btagging", year))
list(cset.keys())

['deepCSV_comb',
 'deepCSV_incl',
 'deepCSV_mujets',
 'deepCSV_shape',
 'deepCSV_wp_values',
 'deepJet_comb',
 'deepJet_incl',
 'deepJet_mujets',
 'deepJet_shape',
 'deepJet_wp_values']

In [8]:
year="2016"
cset = correctionlib.CorrectionSet.from_file(get_pog_json("muon", year))
list(cset.keys())

['NUM_GlobalMuons_DEN_genTracks',
 'NUM_HighPtID_DEN_TrackerMuons',
 'NUM_HighPtID_DEN_genTracks',
 'NUM_IsoMu24_or_IsoTkMu24_DEN_CutBasedIdTight_and_PFIsoTight',
 'NUM_LooseID_DEN_TrackerMuons',
 'NUM_LooseID_DEN_genTracks',
 'NUM_LooseRelIso_DEN_LooseID',
 'NUM_LooseRelIso_DEN_MediumID',
 'NUM_LooseRelIso_DEN_MediumPromptID',
 'NUM_LooseRelIso_DEN_TightIDandIPCut',
 'NUM_LooseRelTkIso_DEN_HighPtIDandIPCut',
 'NUM_LooseRelTkIso_DEN_TrkHighPtIDandIPCut',
 'NUM_MediumID_DEN_TrackerMuons',
 'NUM_MediumID_DEN_genTracks',
 'NUM_MediumPromptID_DEN_TrackerMuons',
 'NUM_MediumPromptID_DEN_genTracks',
 'NUM_Mu50_or_TkMu50_DEN_CutBasedIdGlobalHighPt_and_TkIsoLoose',
 'NUM_SoftID_DEN_TrackerMuons',
 'NUM_SoftID_DEN_genTracks',
 'NUM_TightID_DEN_TrackerMuons',
 'NUM_TightID_DEN_genTracks',
 'NUM_TightRelIso_DEN_MediumID',
 'NUM_TightRelIso_DEN_MediumPromptID',
 'NUM_TightRelIso_DEN_TightIDandIPCut',
 'NUM_TightRelTkIso_DEN_HighPtIDandIPCut',
 'NUM_TightRelTkIso_DEN_TrkHighPtIDandIPCut',
 'NUM_Tra

In [9]:
cset['NUM_MediumPromptID_DEN_TrackerMuons'].evaluate("2016postVFP_UL",[1],[100],"sf")

array([0.9941142])

In [10]:
cset['NUM_MediumPromptID_DEN_TrackerMuons'].evaluate("2016postVFP_UL",[1],[100],"systup")

array([0.99619073])

In [11]:
cset['NUM_MediumPromptID_DEN_TrackerMuons'].evaluate("2016postVFP_UL",[1],[100],"systdown")

array([0.99203768])

In [12]:
jec_compiled = "data/jec_compiled_run2.pkl.gz"

In [13]:
if jec_compiled is not None:
            with gzip.open(jec_compiled, "rb") as filehandler:
                jmestuff = pickle.load(filehandler)

/tmp/ipykernel_1157/1434799976.py:3: DeprecationWarning: Please use `interp1d` from the `scipy.interpolate` namespace, the `scipy.interpolate.interpolate` namespace is deprecated.
  jmestuff = pickle.load(filehandler)


In [14]:
jmestuff

{'jet_factory': {'2016preVFPmc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd28654f820>,
  '2016preVFPmcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd285192800>,
  '2016postVFPmc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2851fe650>,
  '2016postVFPmcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd283dff8e0>,
  '2017mc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd283c6f7f0>,
  '2017mcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2827523e0>,
  '2018mc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2827bee30>,
  '2018mcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2812b7c70>},
 'fatjet_factory': {'2016preVFPmc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd281127b80>,
  '2016preVFPmcNOJER': <coffea.jetmet_tools.CorrectedJetsFa

In [15]:
jet_factory = {}
met_factory = None

jet_factory["ak4"] = jmestuff["jet_factory"]
jet_factory["ak8"] = jmestuff["fatjet_factory"]
met_factory = jmestuff["met_factory"]

In [16]:
jet_factory["ak8"]

{'2016preVFPmc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd281127b80>,
 '2016preVFPmcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd27fd92c80>,
 '2016postVFPmc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd27fdfe980>,
 '2016postVFPmcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd27e879a50>,
 '2017mc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd27e8e1750>,
 '2017mcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd27d3cc250>,
 '2018mc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd27d238b50>,
 '2018mcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd27bd3d780>}

In [17]:
jet_factory["ak4"]

{'2016preVFPmc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd28654f820>,
 '2016preVFPmcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd285192800>,
 '2016postVFPmc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2851fe650>,
 '2016postVFPmcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd283dff8e0>,
 '2017mc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd283c6f7f0>,
 '2017mcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2827523e0>,
 '2018mc': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2827bee30>,
 '2018mcNOJER': <coffea.jetmet_tools.CorrectedJetsFactory.CorrectedJetsFactory at 0x7fd2812b7c70>}

In [19]:
if jec_compiled is not None:
        with gzip.open(jec_compiled, "rb") as filehandler:
            jmestuff = pickle.load(filehandler)

AttributeError: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from '/opt/conda/lib/python3.10/site-packages/cloudpickle/cloudpickle.py'>

In [23]:
jec_compiled = "data/jec_compiled.pkl.gz"
with gzip.open(jec_compiled) as fin:
    compiled = pickle.load(fin)

AttributeError: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from '/opt/conda/lib/python3.10/site-packages/cloudpickle/cloudpickle.py'>